In [4]:
import spacy
from spacy import displacy

In [5]:
spacy.cli.download("en_core_web_sm")
NER = spacy.load("en_core_web_sm")

def spacy_large_ner(document):
  return {(ent.text.strip(), ent.label_) for ent in NER(document).ents}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 18.3 MB/s eta 0:00:0000:0100:01



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3.11 install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
doc="The World Health Organization (WHO)[1] is a specialized agency of the United Nations responsible for international public health.[2] The WHO Constitution states its main objective as 'the attainment by all peoples of the highest possible level of health'.[3] Headquartered in Geneva, Switzerland, it has six regional offices and 150 field offices worldwide. The WHO was established on 7 April 1948.[4][5] The first meeting of the World Health Assembly (WHA), the agency's governing body, took place on 24 July of that year. The WHO incorporated the assets, personnel, and duties of the League of Nations' Health Organization and the Office International d'Hygiène Publique, including the International Classification of Diseases (ICD).[6] Its work began in earnest in 1951 after a significant infusion of financial and technical resources.[7]"

In [10]:
spacy_large_ner("Not coercive ; not involving coercion")

set()

In [12]:
import json

def get_data(filename: str)->list :
    f = open(filename)
    data = json.load(f)
    f.close()
    return data


In [14]:
model_aware_data= get_data("./train.model-aware.v2.json")
types=[]
for item in model_aware_data:
    types.append(item["task"])

print(set(types))

{'PG', 'MT', 'DM'}


In [19]:
dm_data=[item for item in model_aware_data if item["task"]=="DM"]
dm_data[:4]


[{'hyp': 'Of or pertaining to the language of a particular area , or to a particular',
  'tgt': 'Of or pertaining to everyday language , as opposed to standard , literary , liturgical , or scientific idiom .',
  'src': 'There are blacktips , silvertips , bronze whalers , black whalers , spinner sharks , and bignose sharks . these of course are vernacular names , but this is one case where the scientific nomenclature does not clarify the species , since it is now being revised . What is the meaning of vernacular ?',
  'ref': 'tgt',
  'task': 'DM',
  'model': 'ltg/flan-t5-definition-en-base'},
 {'hyp': 'Not coercive ; not involving coercion',
  'tgt': 'Not coercive ; free of coercion',
  'src': 'Mr. obama signed executive orders requiring all interrogations to follow the noncoercive methods of the army field manual . What is the meaning of noncoercive ?',
  'ref': 'tgt',
  'task': 'DM',
  'model': 'ltg/flan-t5-definition-en-base'},
 {'hyp': 'To express or express by words ; to express by

In [29]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

# Ignore: trying to  figure out a way to test hyp with src
def semantic_coherence_check(src, hyp):
    src_tokens = word_tokenize(src.lower())
    hyp_tokens = word_tokenize(hyp.lower())

    stop_words = set(stopwords.words('english'))
    src_filtered = [word for word in src_tokens if word not in stop_words]
    hyp_filtered = [word for word in hyp_tokens if word not in stop_words]

    src_synsets = set(wordnet.synsets(src))
    hyp_synsets = set(wordnet.synsets(hyp))

    semantic_similarity = len(src_synsets.intersection(hyp_synsets)) > 0

    return semantic_similarity

src = "Stop talking cobblers. load of old cobblers. What is the meaning of cobblers?"
hyp = "Nothing"
print("Semantic Coherence Check:", semantic_coherence_check(src, hyp))


Semantic Coherence Check: False


In [31]:
pip install unbabel-comet

  Obtaining dependency information for unbabel-comet from https://files.pythonhosted.org/packages/6e/a1/ee4d2a587c4bcc2de344c1838353d0604bc3e905d5725920dab2218fb4a3/unbabel_comet-2.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for entmax<2.0,>=1.1 from https://files.pythonhosted.org/packages/06/a0/71747f0d98e441d0670b06205afd24d832e88c0ee62129ca47ce88505304/entmax-1.3-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/ab/28/d4b691840d73126d4c9845f8a22dad033ac872509b6d3a0d93b456eef424/huggingface_hub-0.21.4-py3-none-any.whl.metadata
  Obtaining dependency information for jsonargparse==3.13.1 from https://files.pythonhosted.org/packages/04/cf/b8bdb5d6b631766a2332dca19cb2b214620a65c398636012dde69b71f36f/jsonargparse-3.13.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for protobuf<

In [32]:
import comet

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(model_path)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 127875.12it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.2.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
Encoder model frozen.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:188: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [35]:
mt_data=[item for item in model_aware_data if item["task"]=="MT"]
mt_data[:4]

[{'hyp': 'These plants get their food from the sun through photosynthesis, and they also give shade.',
  'tgt': 'Plants make their food from the sun by photosynthesis. They also provide shade.',
  'src': 'Үсемлекләр азыкны фотосинтез юлы белән кояштан ала. Шулай ук алар күләгә дә ясый.',
  'ref': 'either',
  'task': 'MT',
  'model': 'facebook/nllb-200-distilled-600M'},
 {'hyp': 'Only the Stanley mining bank around the FIC western marketplace can exchange money.',
  'tgt': 'Money can be exchanged at the only bank in the islands which is located in Stanley across from the FIC West store.',
  'src': 'Ilé ìfowópamọ́sí tó wà ní erékùsù ní Stanley káàkiri ibi ìtajà ìwọ̀ oòrùn FIC nìkan ni o ti lè pààrọ̀ owó.',
  'ref': 'either',
  'task': 'MT',
  'model': 'facebook/nllb-200-distilled-600M'},
 {'hyp': 'The archipelago is located 120 km north of the island, and the settlement of Villa Las Estrellas is King George Island.',
  'tgt': 'The archipelago lies 120 km north of the Peninsula. The large

In [42]:
data=[]
for item in mt_data:
    data.append({
        "src":item["src"],
        "mt":item["hyp"],
        "ref":item["tgt"]

    })


model_output = model.predict(data, batch_size=8, gpus=0)
print(model_output)
print(model_output.scores) 
print(model_output.system_score) 



GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
Predicting DataLoader 0: 100%|██████████| 1250/1250 [20:21<00:00,  1.02it/s]

Prediction([('scores', [0.8166673183441162, 0.5283975601196289, 0.7975559234619141, 0.5926758050918579, 0.5371964573860168, 0.9012420177459717, 0.6930860280990601, 0.91974276304245, 0.6827398538589478, 0.8103275299072266, 0.5304718017578125, 0.48074930906295776, 0.7977539300918579, 0.634089469909668, 0.5690450668334961, 0.8720316290855408, 0.20398706197738647, 0.7547087073326111, 0.8296449184417725, 0.6541458368301392, 0.8690156936645508, 0.7620254755020142, 0.5490186214447021, 0.9212234616279602, 0.7572113275527954, 0.3824431002140045, 0.6014841794967651, 0.7256578207015991, 0.6137659549713135, 0.4850556254386902, 0.8589468002319336, 0.41278403997421265, 0.9085028767585754, 0.73343825340271, 0.48436158895492554, 0.3465002477169037, 0.8310221433639526, 0.5797715187072754, 0.836890459060669, 0.5640498399734497, 0.7475286722183228, 0.852411150932312, 0.9068681001663208, 0.5507766008377075, 0.7952799201011658, 0.7619296312332153, 0.7273770570755005, 0.4936087727546692, 0.7172704935073853,

In [43]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for datasets>=2.0.0 from https://files.pythonhosted.org/packages/95/fc/661a7f06e8b7d48fcbd3f55423b7ff1ac3ce59526f146fda87a1e1788ee4/datasets-2.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for dill from https://files.pythonhosted.org/packages/c9/7a/cef76fd8438a42f96db64ddaa85280485a9c395e7df3db8158cfec1eee34/dill-0.3.8-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/9c/0a/d0fd8d78c8a2c3c3b34e7a9dccf85f01bf38f32e0228d107fa3903e0981f/xxhash-3.4.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for multiprocess from https://files.pythonhosted.org/packages/50/15/b56e50e8debaf439f44befec5b2af11db85f6e0f344c3113ae0be0593a91/multiprocess-0.70.1

In [46]:
import evaluate
evaluate.list_evaluation_modules()

PyTorch version 2.2.1 available.
TensorFlow version 2.15.0 available.


['lvwerra/test',
 'jordyvl/ece',
 'angelina-wang/directional_bias_amplification',
 'cpllab/syntaxgym',
 'lvwerra/bary_score',
 'hack/test_metric',
 'yzha/ctc_eval',
 'codeparrot/apps_metric',
 'mfumanelli/geometric_mean',
 'daiyizheng/valid',
 'erntkn/dice_coefficient',
 'mgfrantz/roc_auc_macro',
 'Vlasta/pr_auc',
 'gorkaartola/metric_for_tp_fp_samples',
 'idsedykh/metric',
 'idsedykh/codebleu2',
 'idsedykh/codebleu',
 'idsedykh/megaglue',
 'cakiki/ndcg',
 'Vertaix/vendiscore',
 'GMFTBY/dailydialogevaluate',
 'GMFTBY/dailydialog_evaluate',
 'jzm-mailchimp/joshs_second_test_metric',
 'ola13/precision_at_k',
 'yulong-me/yl_metric',
 'abidlabs/mean_iou',
 'abidlabs/mean_iou2',
 'KevinSpaghetti/accuracyk',
 'NimaBoscarino/weat',
 'ronaldahmed/nwentfaithfulness',
 'Viona/infolm',
 'kyokote/my_metric2',
 'kashif/mape',
 'Ochiroo/rouge_mn',
 'giulio98/code_eval_outputs',
 'leslyarun/fbeta_score',
 'giulio98/codebleu',
 'anz2/iliauniiccocrevaluation',
 'zbeloki/m2',
 'xu1998hz/sescore',
 'dvit

In [54]:

import sys
import math
import unicodedata
import argparse
from collections import defaultdict
import time
import string

def separate_characters(line):
    return list(line.strip().replace(" ", ""))

def separate_punctuation(line):
    words = line.strip().split()
    tokenized = []
    for w in words:
        if len(w) == 1:
            tokenized.append(w)
        else:
            lastChar = w[-1] 
            firstChar = w[0]
            if lastChar in string.punctuation:
                tokenized += [w[:-1], lastChar]
            elif firstChar in string.punctuation:
                tokenized += [firstChar, w[1:]]
            else:
                tokenized.append(w)
    
    return tokenized
    
def ngram_counts(wordList, order):
    counts = defaultdict(lambda: defaultdict(float))
    nWords = len(wordList)
    for i in range(nWords):
        for j in range(1, order+1):
            if i+j <= nWords:
                ngram = tuple(wordList[i:i+j])
                counts[j-1][ngram]+=1
   
    return counts

def ngram_matches(ref_ngrams, hyp_ngrams):
    matchingNgramCount = defaultdict(float)
    totalRefNgramCount = defaultdict(float)
    totalHypNgramCount = defaultdict(float)
 
    for order in ref_ngrams:
        for ngram in hyp_ngrams[order]:
            totalHypNgramCount[order] += hyp_ngrams[order][ngram]
        for ngram in ref_ngrams[order]:
            totalRefNgramCount[order] += ref_ngrams[order][ngram]
            if ngram in hyp_ngrams[order]:
                matchingNgramCount[order] += min(ref_ngrams[order][ngram], hyp_ngrams[order][ngram])


    return matchingNgramCount, totalRefNgramCount, totalHypNgramCount


def ngram_precrecf(matching, reflen, hyplen, beta):
    ngramPrec = defaultdict(float)
    ngramRec = defaultdict(float)
    ngramF = defaultdict(float)
    
    factor = beta**2
    
    for order in matching:
        if hyplen[order] > 0:
            ngramPrec[order] = matching[order]/hyplen[order]
        else:
            ngramPrec[order] = 1e-16
        if reflen[order] > 0:
            ngramRec[order] = matching[order]/reflen[order]
        else:
            ngramRec[order] = 1e-16
        denom = factor*ngramPrec[order] + ngramRec[order]
        if denom > 0:
            ngramF[order] = (1+factor)*ngramPrec[order]*ngramRec[order] / denom
        else:
            ngramF[order] = 1e-16
            
    return ngramF, ngramRec, ngramPrec


def computeChrF(data, nworder=6, ncorder=2, beta=2, sentence_level_scores = None):
    norder = float(nworder + ncorder)

    # initialisation of document level scores
    totalMatchingCount = defaultdict(float)
    totalRefCount = defaultdict(float)
    totalHypCount = defaultdict(float)
    totalChrMatchingCount = defaultdict(float)
    totalChrRefCount = defaultdict(float)
    totalChrHypCount = defaultdict(float)
    averageTotalF = 0.0

    nsent = 0
    for item in data:
        hyp,ref=(item["hyp"], item["tgt"])
        nsent += 1
        
        # preparation for multiple references
        maxF = 0.0
        bestWordMatchingCount = None
        bestCharMatchingCount = None
        
        hypNgramCounts = ngram_counts(separate_punctuation(hyp), nworder)
        hypChrNgramCounts = ngram_counts(separate_characters(hyp), ncorder)
        refNgramCounts = ngram_counts(separate_punctuation(ref), nworder)
        refChrNgramCounts = ngram_counts(separate_characters(ref), ncorder)

        refNgramCounts = ngram_counts(separate_punctuation(ref), nworder)
        refChrNgramCounts = ngram_counts(separate_characters(ref), ncorder)

            # number of overlapping n-grams, total number of ref n-grams, total number of hyp n-grams
        matchingNgramCounts, totalRefNgramCount, totalHypNgramCount = ngram_matches(refNgramCounts, hypNgramCounts)
        matchingChrNgramCounts, totalChrRefNgramCount, totalChrHypNgramCount = ngram_matches(refChrNgramCounts, hypChrNgramCounts)
                    
            # n-gram f-scores, recalls and precisions
        ngramF, ngramRec, ngramPrec = ngram_precrecf(matchingNgramCounts, totalRefNgramCount, totalHypNgramCount, beta)
        chrNgramF, chrNgramRec, chrNgramPrec = ngram_precrecf(matchingChrNgramCounts, totalChrRefNgramCount, totalChrHypNgramCount, beta)

        sentRec  = (sum(chrNgramRec.values())  + sum(ngramRec.values()))  / norder
        sentPrec = (sum(chrNgramPrec.values()) + sum(ngramPrec.values())) / norder
        sentF    = (sum(chrNgramF.values())    + sum(ngramF.values()))    / norder

        if sentF > maxF:
            maxF = sentF
            bestMatchingCount = matchingNgramCounts
            bestRefCount = totalRefNgramCount
            bestHypCount = totalHypNgramCount
            bestChrMatchingCount = matchingChrNgramCounts
            bestChrRefCount = totalChrRefNgramCount
            bestChrHypCount = totalChrHypNgramCount


        # collect document level ngram counts
        for order in range(nworder):
            totalMatchingCount[order] += bestMatchingCount[order]
            totalRefCount[order] += bestRefCount[order]
            totalHypCount[order] += bestHypCount[order]
        for order in range(ncorder):
            totalChrMatchingCount[order] += bestChrMatchingCount[order]
            totalChrRefCount[order] += bestChrRefCount[order]
            totalChrHypCount[order] += bestChrHypCount[order]

        averageTotalF += maxF

    # all sentences are done
     
    totalNgramF, totalNgramRec, totalNgramPrec = ngram_precrecf(totalMatchingCount, totalRefCount, totalHypCount, beta)
    totalChrNgramF, totalChrNgramRec, totalChrNgramPrec = ngram_precrecf(totalChrMatchingCount, totalChrRefCount, totalChrHypCount, beta)

    totalF    = (sum(totalChrNgramF.values())    + sum(totalNgramF.values()))    / norder
    averageTotalF = averageTotalF / nsent
    totalRec  = (sum(totalChrNgramRec.values())  + sum(totalNgramRec.values()))  / norder
    totalPrec = (sum(totalChrNgramPrec.values()) + sum(totalNgramPrec.values())) / norder

    return totalF, averageTotalF, totalPrec, totalRec

In [56]:
computeChrF(mt_data)

(0.3265904635568851,
 0.3256036095548835,
 0.32269027559865127,
 0.3276160325797506)